# pandas lesson 

Objective: Wrangle Data with pandas package.  Create a dataset, add new columns, select columns, filter rows group and aggregate.

 A pandas Dataframe is a 2D object (table).
 A pandas Series is a 1D labelled object.
 pandas rows have unique labels (a strange concept to SQL people).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # pandas uses matplotlib for plotting

In [ ]:
#  Let's have a quick look at a series
p = pd.Series(np.random.randn(4), name = 'price')
p, p.index, p.values, p.dtype, p.shape, p.ndim, p.size

In [ ]:
p * 100 # multiply all values by 100, an element-wise operation

In [ ]:
np.abs(p) # absolute value of all elements

In [ ]:
p.mean(), p.std(), p.min(), p.max(), p.median(), p.quantile(0.25), p.quantile(0.75) # some summary statistics

In [ ]:
p.index = ["HSBC", "BP", "TSCO", "RDSA"] 
#p.describe()
p

In [ ]:
p['BP'] # get a value in the Series


In [ ]:
p['BP'] = 5.0 # set a value in the Series
p['BP']

In [ ]:
'TSCO' in p # is this value a member of the Series

In [ ]:
#  We can build a pandas dataframe from a dict
fb_dict = {
        'id' : ['MCY', 'LIV', 'TOT', 'CHE', 'ARL'],
        'city' :	['Manchester',	'Liverpool', 'London', 'London', 'London'],
        'team' :	['Manchester City', 'Liverpool', 'Tottenham Hotspur', 'Chelsea', 'Arsenal'],
        'won' :	[5, 6, 6, 5, 5],
        'drawn' : [4, 1, 0, 2,0],
        'lost' : [0, 0, 2, 0, 2],
        'form' : ['DWWWW', 'WWWWD', 'LLWWW', 'WWWDD', 'WWWWW']
        }
#  Create a pandas dataframe
# The dict's keys become the column names and the dict's values become the column values
fb_df = pd.DataFrame(fb_dict)

# set the index to the unique values of the 'id' column - more useful than 0,1,2...
fb = fb_df.set_index('id')
fb


In [ ]:
fb.index

In [ ]:
fb.head(2), fb.tail(2)


In [ ]:
fb.describe() # summary statistics of the numeric columns

In [ ]:
fb.shape

In [ ]:
fb.values # a numpy array

In [ ]:
# fb[['city']]
fb.city # values of a column.  Note the  dot syntax
fb.city.unique()


In [ ]:

fb.columns #  Return a list of the columns

In [ ]:
# Create new columns
# Teams get 3 point for a win, 1 for a draw, none for a loss
fb['points'] = fb['won'] * 3 + fb['drawn']
fb
